In [1]:
%pip install transformers datasets tokenizers seqeval torch torchvision torchaudio -q
%pip install accelerate==0.20.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 5.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer, pipeline

In [3]:
import json

with open("annotations.json", "r") as json_file:
  data = json.load(json_file)

In [4]:
data['annotations'][0][1]['entities'][0:10]

[[18, 26, 'B-ORG'],
 [27, 34, 'I-ORG'],
 [35, 41, 'B-MISC'],
 [42, 46, 'I-MISC'],
 [47, 51, 'I-MISC'],
 [52, 55, 'I-MISC'],
 [58, 62, 'B-LOC'],
 [63, 68, 'I-LOC'],
 [69, 74, 'I-LOC'],
 [75, 78, 'I-LOC']]

## Transform annotations to huggingface datadict

In [5]:
text = data["annotations"][0][0]
tags = data["annotations"][0][1]["entities"]
classes = data["classes"]

classes_dict = {}
for i in range(len(classes)):
  classes_dict[classes[i]] = i

# print(classes_dict)

tokens = []
named_tags = []

for tag in tags:
  tokens.append(text[tag[0]:tag[1]])
  named_tags.append(tag[2])


anno_dict = dict(zip(tokens, named_tags))
print(anno_dict)

all_tokens = text.split(" ")
named_labels = []
numbered_labels = []

for token in all_tokens:
  if token not in anno_dict:
    named_labels.append("NONE")
  else:
    named_labels.append(anno_dict[token])

for label in named_labels:
  numbered_labels.append(classes_dict[label])

print(all_tokens)
# print(named_labels)
print(numbered_labels)

{'Inveniam': 'B-ORG', 'Private': 'I-ORG', 'Equity': 'B-MISC', 'Fund': 'I-ORG', 'Demo': 'I-MISC', 'IV.': 'I-MISC', '8500': 'B-LOC', 'World': 'I-LOC', 'Trade': 'I-LOC', 'New': 'B-LOC', 'York': 'I-LOC', '10022': 'I-LOC', 'January': 'B-DATE', '1': 'I-DATE', '2019': 'I-DATE', 'Maria': 'B-PER', 'Sharapova': 'B-PER', '31': 'B-LOC', 'Blandford': 'I-LOC', 'Street': 'I-LOC', 'London': 'I-LOC', 'IV': 'I-MISC', 'Delaware': 'B-ORG', 'limited': 'B-MISC', 'partnership': 'I-AGRE', 'letter': 'B-AGRE', 'agreement': 'I-AGRE', '"Letter': 'B-AGRE', 'Agreement"': 'I-AGRE', '"Investor"': 'B-MISC', 'Amended': 'B-AGRE', 'Restated': 'I-AGRE', 'Limited': 'B-MISC', 'Partnership': 'I-AGRE', 'Agreement': 'I-AGRE', 'August': 'B-DATE', '27': 'I-DATE', '2021': 'I-DATE', '"Limited': 'B-AGRE', 'Subscription': 'I-AGRE', 'Investor': 'B-MISC', '"Subscription': 'B-AGRE', '$': 'B-CURR', '50,000,000': 'I-CURR', '"Capital': 'B-AGRE', 'Commitment"': 'I-AGRE', 'Capital': 'I-MISC', 'Partners': 'I-MISC', 'GP': 'I-ORG', 'L.L.C.': '

In [6]:
all_tokens_array = []
numbered_labels_array = []

temp1 = []
temp2 = []
for i in range(len(all_tokens)):
  temp1.append(all_tokens[i])
  temp2.append(numbered_labels[i])
  if i%10 == 0 and i != 0:
    all_tokens_array.append(temp1)
    numbered_labels_array.append(temp2)
    temp1 = []
    temp2 = []
all_tokens_array.append(temp1)
numbered_labels_array.append(temp2)


print(all_tokens_array)
print(numbered_labels_array)

[['EXECUTION', 'VERSION', 'Inveniam', 'Private', 'Equity', 'Fund', 'Demo', 'IV.', '.', '8500', 'World'], ['Trade', 'New', 'York,', 'New', 'York', '10022', 'January', '1,', '2019', 'Maria'], ['Sharapova', '31', 'Blandford', 'Street', 'London,', 'W1U', '3DN', 'Ladies', 'and', 'Gentlemen:'], ['RE:', 'Inveniam', 'Private', 'Equity', 'Fund', 'Demo', 'IV,', 'a', 'Delaware', 'limited'], ['partnership', '(the', '"Fund")', 'This', 'letter', 'agreement', '(this', '"Letter', 'Agreement")', 'is'], ['entered', 'into', 'in', 'connection', 'with', 'the', 'purchase', 'by', 'Sharapova', '(the'], ['"Investor")', 'of', 'a', 'limited', 'partnership', 'interest', 'in', 'the', 'Fund.', 'Capitalized'], ['terms', 'used', 'and', 'not', 'defined', 'herein', 'shall', 'have', 'the', 'meanings'], ['given', 'to', 'them', 'in', 'the', 'Fourth', 'Amended', 'and', 'Restated', 'Limited'], ['Partnership', 'Agreement', 'of', 'the', 'Fund', 'dated', 'as', 'of', 'August', '27,'], ['2021', '(the', '"Limited', 'Partnership',

In [7]:
print(classes_dict)

{'NONE': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-DATE': 3, 'I-DATE': 4, 'B-TIME': 5, 'I-TIME': 6, 'B-PER': 7, 'I-PER': 8, 'B-CURR': 9, 'I-CURR': 10, 'B-LOC': 11, 'I-LOC': 12, 'B-AGRE': 13, 'I-AGRE': 14, 'B-MISC': 15, 'I-MISC': 16}


In [8]:
data_dict_train = {
    "tokens" : all_tokens_array[0:200],
    "labels" : numbered_labels_array[0:200]
}

data_dict_val = {
    "tokens" : all_tokens_array[200:225],
    "labels" : numbered_labels_array[200:225]
}

data_dict_test = {
    "tokens" : all_tokens_array[225:],
    "labels" : numbered_labels_array[225:]
}

dataset_train = Dataset.from_dict(data_dict_train)
dataset_val = Dataset.from_dict(data_dict_val)
dataset_test = Dataset.from_dict(data_dict_test)

dataset_dict = DatasetDict({"train": dataset_train, "validate": dataset_val, "test": dataset_test})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 200
    })
    validate: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 25
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 25
    })
})


In [9]:
dataset_dict['train'][11]

{'tokens': ['Investor',
  'and',
  'the',
  'Fund',
  'dated',
  'as',
  'of',
  'the',
  'date',
  'hereof'],
 'labels': [15, 0, 0, 2, 0, 0, 0, 0, 0, 0]}

### Modeling

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [11]:
example_tokens = dataset_dict["train"][11]
inputs = tokenizer(example_tokens["tokens"], is_split_into_words=True,)
print("Miss Matched Length!!")
print(len(inputs.tokens()))
print(len(example_tokens["labels"]))

Miss Matched Length!!
13
10


In [12]:
inputs = tokenizer(dataset_dict["train"][1]["tokens"], is_split_into_words=True,)
print(inputs.tokens())
print(dataset_dict["train"][1]["tokens"])
print(dataset_dict['train'][1]['labels'])
print(inputs.word_ids())

['[CLS]', 'trade', 'new', 'york', ',', 'new', 'york', '100', '##22', 'january', '1', ',', '2019', 'maria', '[SEP]']
['Trade', 'New', 'York,', 'New', 'York', '10022', 'January', '1,', '2019', 'Maria']
[12, 11, 0, 11, 12, 12, 3, 0, 4, 7]
[None, 0, 1, 2, 2, 3, 4, 5, 5, 6, 7, 7, 8, 9, None]


In [13]:
def align_labels_with_tokens(labels, word_ids):
    for label in labels:
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])
            else:
                label_ids.append(labels[word_idx])
            previous_word_idx = word_idx

        return label_ids

In [14]:
labels = dataset_dict["train"][11]["labels"]
inputs = tokenizer(dataset_dict["train"][11]["tokens"], is_split_into_words=True,)
word_ids = inputs.word_ids()
print(dataset_dict["train"][11]["tokens"])
print(align_labels_with_tokens(labels, word_ids))
print(labels)

['Investor', 'and', 'the', 'Fund', 'dated', 'as', 'of', 'the', 'date', 'hereof']
[-100, 15, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, -100]
[15, 0, 0, 2, 0, 0, 0, 0, 0, 0]


In [15]:
example_tokens = dataset_dict["train"][11]
inputs = tokenizer(example_tokens["tokens"], is_split_into_words=True,)
print("Miss Matched Length!!")
print(len(inputs.tokens()))
print(len(example_tokens["labels"]))
print("Label matching successful!!")
print(len(align_labels_with_tokens(example_tokens["labels"], inputs.word_ids())))

Miss Matched Length!!
13
10
Label matching successful!!
13


In [16]:
def tokenizer_function(dataset):
  tokenized_dataset = tokenizer(dataset['tokens'],truncation=True,is_split_into_words=True,)
  tokenized_dataset['labels'] = align_labels_with_tokens(dataset['labels'],tokenized_dataset.word_ids())
  return tokenized_dataset

In [17]:
dataset_dict['train'][15]

{'tokens': ['$50,000,000',
  'as',
  'of',
  'the',
  'date',
  'hereof',
  '(the',
  '"Capital',
  'Commitment"),',
  'Inveniam'],
 'labels': [0, 0, 0, 0, 0, 0, 0, 13, 0, 1]}

In [18]:
# print(dataset_dict)
tokenized_dataset = dataset_dict.map(tokenizer_function, remove_columns=['tokens'])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [19]:
# print(len(tokenized_dataset['train'][15]['input_ids']))
# print(len(tokenized_dataset['train'][15]['labels']))

In [20]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=17)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: 

In [21]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [22]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [23]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["validate"],
   data_collator=data_collator,
   tokenizer=tokenizer,
)

In [24]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.935018
2,No log,0.701018
3,No log,0.573947
4,No log,0.519404
5,No log,0.508062


TrainOutput(global_step=100, training_loss=0.7682083892822266, metrics={'train_runtime': 314.0386, 'train_samples_per_second': 3.184, 'train_steps_per_second': 0.318, 'total_flos': 9514125124320.0, 'train_loss': 0.7682083892822266, 'epoch': 5.0})

## Preparing the fine-tuned model

In [26]:
model.save_pretrained("ner_model")

In [27]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [28]:
classes

['NONE',
 'B-ORG',
 'I-ORG',
 'B-DATE',
 'I-DATE',
 'B-TIME',
 'I-TIME',
 'B-PER',
 'I-PER',
 'B-CURR',
 'I-CURR',
 'B-LOC',
 'I-LOC',
 'B-AGRE',
 'I-AGRE',
 'B-MISC',
 'I-MISC']

In [29]:
id2label = {
    str(i): label for i,label in enumerate(classes)
}
label2id = {
    label: str(i) for i,label in enumerate(classes)
}

In [30]:
id2label

{'0': 'NONE',
 '1': 'B-ORG',
 '2': 'I-ORG',
 '3': 'B-DATE',
 '4': 'I-DATE',
 '5': 'B-TIME',
 '6': 'I-TIME',
 '7': 'B-PER',
 '8': 'I-PER',
 '9': 'B-CURR',
 '10': 'I-CURR',
 '11': 'B-LOC',
 '12': 'I-LOC',
 '13': 'B-AGRE',
 '14': 'I-AGRE',
 '15': 'B-MISC',
 '16': 'I-MISC'}

In [31]:
import json
config = json.load(open("ner_model/config.json"))

In [32]:
config["id2label"] = id2label
config["label2id"] = label2id

In [33]:
json.dump(config, open("ner_model/config.json","w"))

In [34]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

### Testing

In [35]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = '''Third Party Investor is a Comparable Investor, the investment of such Third Party Investor in the Fund, the Sister Fund or the SFRO Feeder may'''

ner_results = nlp(example)

print(ner_results)

[{'entity': 'B-MISC', 'score': 0.7002061, 'index': 1, 'word': 'third', 'start': 0, 'end': 5}, {'entity': 'I-MISC', 'score': 0.5671134, 'index': 2, 'word': 'party', 'start': 6, 'end': 11}, {'entity': 'B-MISC', 'score': 0.5713753, 'index': 3, 'word': 'investor', 'start': 12, 'end': 20}, {'entity': 'NONE', 'score': 0.9794135, 'index': 4, 'word': 'is', 'start': 21, 'end': 23}, {'entity': 'NONE', 'score': 0.97981066, 'index': 5, 'word': 'a', 'start': 24, 'end': 25}, {'entity': 'B-MISC', 'score': 0.47286883, 'index': 6, 'word': 'comparable', 'start': 26, 'end': 36}, {'entity': 'B-MISC', 'score': 0.5818263, 'index': 7, 'word': 'investor', 'start': 37, 'end': 45}, {'entity': 'NONE', 'score': 0.97984743, 'index': 8, 'word': ',', 'start': 45, 'end': 46}, {'entity': 'NONE', 'score': 0.98178655, 'index': 9, 'word': 'the', 'start': 47, 'end': 50}, {'entity': 'B-MISC', 'score': 0.35250932, 'index': 10, 'word': 'investment', 'start': 51, 'end': 61}, {'entity': 'NONE', 'score': 0.9772727, 'index': 11,